In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model
from keras.layers import *
from keras import backend as K
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50, preprocess_input
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
x_train = np.load('TrainingData/X_train.npy')
y_train = np.load('TrainingData/Y_train.npy')

x_test = np.load('TestCases/X_test.npy')

In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(1252, 227, 227, 3)
(1252,)
(313, 227, 227, 3)


In [14]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)

In [4]:
model = MobileNet(input_shape=(227,227,3), include_top=False, weights=None)

W0806 00:00:14.628401 139742901028672 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 00:00:14.652484 139742901028672 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 00:00:14.663264 139742901028672 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 00:00:14.712238 139742901028672 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 228, 228, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 113, 113, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 113, 113, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 113, 113, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 113, 113, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 113, 113, 32)      128       
__________

In [6]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(10,activation='softmax')(d1)

model_new = Model(input = model.input, outputs = fc2)

W0806 00:00:24.247421 139742901028672 deprecation.py:506] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model_new.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 228, 228, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 113, 113, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 113, 113, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 113, 113, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 113, 113, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 113, 113, 32)      128       
__________

In [11]:
# adam = Adam(lr-0.00003)
model_new.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0806 00:08:34.928034 139742901028672 deprecation_wrapper.py:119] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
for ix in range(len(model_new.layers)):
    print(ix, model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f182c56fc88>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f1859ecc320>
2 <keras.layers.convolutional.Conv2D object at 0x7f1859ecc358>
3 <keras.layers.normalization.BatchNormalization object at 0x7f185a727ba8>
4 <keras.layers.advanced_activations.ReLU object at 0x7f1859ecc470>
5 <keras.layers.convolutional.DepthwiseConv2D object at 0x7f182c512588>
6 <keras.layers.normalization.BatchNormalization object at 0x7f182c512c50>
7 <keras.layers.advanced_activations.ReLU object at 0x7f17f2204e48>
8 <keras.layers.convolutional.Conv2D object at 0x7f17f00b1d68>
9 <keras.layers.normalization.BatchNormalization object at 0x7f17f0053e10>
10 <keras.layers.advanced_activations.ReLU object at 0x7f17f00745f8>
11 <keras.layers.convolutional.ZeroPadding2D object at 0x7f17e87b9f98>
12 <keras.layers.convolutional.DepthwiseConv2D object at 0x7f17e8741d68>
13 <keras.layers.normalization.BatchNormalization object at 0x7f17e8734860>
14 <keras.layers.

In [15]:
hist = model_new.fit(x_train,y_train,shuffle=True,batch_size=10,epochs=10,validation_split=0.20)

W0806 00:16:43.667652 139742901028672 deprecation.py:323] From /home/shubham/.conda/envs/my_env/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1001 samples, validate on 251 samples
Epoch 1/10
1001/1001 [==============================] - 168s 168ms/step - loss: 2.4616 - acc: 0.1598 - val_loss: 3.7153 - val_acc: 0.1673
Epoch 2/10
1001/1001 [==============================] - 159s 159ms/step - loss: 2.0902 - acc: 0.2637 - val_loss: 2.7383 - val_acc: 0.2032
Epoch 3/10
1001/1001 [==============================] - 157s 157ms/step - loss: 1.9160 - acc: 0.3237 - val_loss: 2.7455 - val_acc: 0.2072
Epoch 4/10
1001/1001 [==============================] - 157s 157ms/step - loss: 1.8532 - acc: 0.3497 - val_loss: 2.1097 - val_acc: 0.3865
Epoch 5/10
1001/1001 [==============================] - 157s 157ms/step - loss: 1.7639 - acc: 0.3506 - val_loss: 2.9409 - val_acc: 0.3108
Epoch 6/10
1001/1001 [==============================] - 156s 156ms/step - loss: 1.6984 - acc: 0.4036 - val_loss: 3.6132 - val_acc: 0.2510
Epoch 7/10
1001/1001 [==============================] - 157s 157ms/step - loss: 1.6722 - acc: 0.4046 - val_loss: 2.1148 - val

In [17]:
y_test = model.predict(x_test)

In [18]:
print(y_test)

[[[[0.00000000e+00 5.13250113e+00 6.00000000e+00 ... 6.00000000e+00
    6.00000000e+00 3.32797694e+00]
   [0.00000000e+00 4.24360657e+00 6.00000000e+00 ... 3.90897465e+00
    6.00000000e+00 3.10658836e+00]
   [0.00000000e+00 3.79694390e+00 6.00000000e+00 ... 3.34007406e+00
    6.00000000e+00 1.82247078e+00]
   ...
   [0.00000000e+00 1.31012559e+00 2.91304994e+00 ... 1.89615592e-01
    2.11883402e+00 3.92794728e-01]
   [3.77450317e-01 0.00000000e+00 1.27133615e-02 ... 0.00000000e+00
    9.54553634e-02 0.00000000e+00]
   [5.65279365e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]]

  [[0.00000000e+00 4.32767057e+00 6.00000000e+00 ... 5.68148899e+00
    6.00000000e+00 4.07676792e+00]
   [0.00000000e+00 3.86435366e+00 6.00000000e+00 ... 3.00978470e+00
    6.00000000e+00 3.86655307e+00]
   [0.00000000e+00 3.33508563e+00 6.00000000e+00 ... 1.86377692e+00
    6.00000000e+00 2.88355899e+00]
   ...
   [0.00000000e+00 1.77712262e+00 3.02081966e+00 ... 0.00

In [20]:
ans = y_test

In [27]:
ans = K.argmax(y_test)
alpha = np.argmax(y_test)

print(ans.shape)
print(alpha)

(313, 7, 7)
2


In [28]:
print(x_test.shape)

(313, 227, 227, 3)


In [40]:
np.argmax?

In [30]:
arr = np.load('TrainingData/Y_train.npy')

In [31]:
print(arr)

[4 9 0 ... 3 5 1]


In [33]:
from keras.utils import np_utils

In [35]:
uniques, ids = np.unique(y_test, return_inverse=True)
y_code = np_utils.to_categorical(ids, len(uniques))
uniques[y_code.argmax(1)]

MemoryError: 

In [39]:
from numpy.random import randint
from numpy import argmax
from keras.utils.np_utils import to_categorical
k = 10
n = 313
x = y_test
# print(x)
print(argmax(to_categorical(x, k)))

0
